In [7]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import pandas as pd
from data import get_data
from time import time
import joblib
pd.set_option('display.max_rows', 20, 
              'display.max_columns', 100)

## Create Master Data

In [8]:
def get_data(urls, sheets, types):
    data = ""
    t0 = time()
    a = []
    i = 0
    for s in sheets:
        s = pd.read_excel(urls, sheet_name=s)
        s = s[[types, 'description']]
        s['description'] = s['description'].str.lower()
        if i == 0:
            data = s.copy()
        else:
            data = pd.concat([data, s], ignore_index=True)
        i += 1
    data.columns = ['target', 'data']
    data['target'] = data['target'].apply(str)
    load_time = time() - t0
    print("Load dataset time:  %0.3fs" % load_time)

    return data

In [9]:
def loads_dataset(urls, sheets):
    data = ""
    t0 = time()
    a = []
    i = 0
    for s in sheets:
        df = pd.read_excel(urls, sheet_name=s)
        df = df[[ 'description','section','chapter','heading','sub-heading','tariff']]
        df['section'] = df['section'].map('{:02}'.format)
        df['chapter'] = df['chapter'].map('{:02}'.format)
    #s['heading'] = s['heading'].apply('{:04}'.format)
    #s['sub-heading'] = s['sub-heading'].map('{:06}'.format)
    #s['tariff'] = s['tariff'].map('{:08}'.format)
        df['description'] = df['description'].str.lower()
    
        if i == 0:
            data = df.copy()
        else:
            data = pd.concat([data, df], ignore_index=True)
        i += 1
        print("Load Data.. "+s+" success")
   # data.columns = ['target', 'data']
  #  data['target'] = data['target'].apply(str)
        load_time = time() - t0
        print("Load dataset time:  %0.3fs" % load_time)

    return data   

In [10]:
urls = 'data/test_hs_code.xlsx'
types = 'section'

#sheets = ['8_digit','6_digit','4_digit','2_digit', 'declaration_2019_10', 'exp','imp']
sheets = ['imp']

df = loads_dataset(urls, sheets)
df.sample(10)

ValueError: '=' alignment not allowed in string format specifier

In [5]:
print(len(df))
df.describe()

101409


,heading,sub-heading
count,101313.000000,100092.000000
mean,7014.387344,703597.748332
std,2016.763019,199685.093335
min,101.000000,10121.000000
25%,7219.000000,730429.000000
50%,7326.000000,732690.000000
75%,8481.000000,848180.000000
max,9706.000000,970600.000000


In [6]:
df.drop_duplicates()
len(df)

101409

In [7]:
#save the DataFrame
print("Save DataFrame")

joblib.dump(df, 'data/master_data_02_2021.pkl', compress=1)
print('success')

Save DataFrame
success


## Add data

In [12]:
master_df = joblib.load('data/master_data_02_2021.pkl')
master_df

,description,section,chapter,heading,sub-heading,tariff
0,"horses; live, purebred breeding animals - pure...",01,01,101.0,10121.0,01012100
1,"horses; live, other than purebred breeding ani...",01,01,101.0,10129.0,01012900
2,asses; live - other,01,01,101.0,10130.0,01013090
3,mules and hinnies; live- other,01,01,101.0,10190.0,01019000
4,"cattle; live, purebred breeding animals - pure...",01,01,102.0,10221.0,01022100
...,...,...,...,...,...,...
101404,"wire brush, hand, with wood handle",20,96,9603.0,960350.0,96035000
101405,wypall l20 perforated jumbo roll wiper 2 polls,10,48,4818.0,481810.0,48181000
101406,"yellow tape 2"" x 10 ytd",07,39,3919.0,391990.0,39199090
101407,zinc it crc,06,34,3402.0,340290.0,34029099


In [14]:
urls = 'data/test_hs_code.xlsx'
types = 'section'


s = 'imp'

data =pd.read_excel(urls, sheet_name=s)
data = data[[ 'description','section','chapter','heading','sub-heading','tariff']]
data['section'] = data['section'].map('{:02}'.format)
#data['chapter'] = data['chapter'].map('{:02}'.format)
data['description'] = data['description'].str.lower()
data

,description,section,chapter,heading,sub-heading,tariff
0,"4-1/2"" retrievable packer",15,73,7326,732690,73269099
1,' multi-meter,16,85,8544,854430,85443099
2,x over ht 55 pin & 65/8 fh box : dvpt13409a-2,15,73,7307,730799,73079990
3,!!apg footwear!! men's 6-inch boot brown,12,64,6403,640340,64034000
4,!!apg footwear!! men's 8-inch boot brown,12,64,6403,640340,64034000
...,...,...,...,...,...,...
98706,sleeping tester,18,90,9029,902910,90291090
98707,gem car vin:52cg2sga0d0003886,17,87,8703,870310,87031010
98708,sbf pin,15,73,7318,731823,73182300
98709,sbf sheave,16,84,8425,842542,84254290


In [16]:
df = pd.concat([data, master_df], ignore_index=True)
print(len(df))
df.drop_duplicates()
len(df)

200120


200120

In [17]:
#save the DataFrame
print("Save DataFrame")

joblib.dump(df, 'data/master_data_02_2021.pkl', compress=1)
print('success')

Save DataFrame
success


## Create Model

In [18]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import pandas as pd

from data import get_data
from time import time
import joblib
%config InlineBackend.figure_format='retina'
print(f'pandas version:  {pd.__version__}')
pd.Timestamp.now()

pandas version:  1.1.3


Timestamp('2021-02-05 16:00:23.519582')

In [19]:
master_data = "data/master_data_02_2021.pkl"

In [20]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\n", "", string)    
    string = re.sub(r"\r", "", string) 
    string = re.sub(r"[0-9]", "digit", string)
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    string = re.sub(r"<", "", string)
    return string.strip().lower()

In [21]:
import re 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

def create_model(df,names):
    X = []
    for i in range(len(df)):
        X.append(clean_str(df.iloc[i][1]))
    
    y = np.array(df["target"])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=5)



    print("pipeline of feature engineering and model")
    model = Pipeline([('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

    print("paramater selection")


    parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}

    print("Training: ")
   
    t0 = time()
    gs_clf_svm = GridSearchCV(model, parameters, n_jobs=-1)
    gs_clf_svm = gs_clf_svm.fit(X, y)
    print(gs_clf_svm.best_score_)
    print(gs_clf_svm.best_params_)

    model = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

#fit model with training data
    model.fit(X_train, y_train)

    pred = model.predict(X_test)

    print(accuracy_score(y_test, pred))

#save the model
    print("Save Model")

    joblib.dump(model, 'data/model/model_'+names +'.pkl', compress=1)
    print("Create Model.. "+names+" success")


    print (classification_report(y_test, pred))

In [22]:
import re
def clean_string(string,chars=None):
    """
    Cleans a ticker for easier use throughout MoneyTree

    Splits by space and only keeps first bit. Also removes
    any characters that are not letters. Returns as lowercase. / uppercase

    >>> clean_ticker('^VIX')
    'vix'
    >>> clean_ticker('SPX Index')
    'spx'
    """
    pattern = re.compile('[\W_+,\s]')
    #res = pattern.sub('', ticker.split(' ')[0])
   
    res = pattern.sub('', string)
    #print(res)
    if chars =='upper':
        return res.upper()
    else:
        return res.lower()
  



    def clean_strings(strings):
        """
        Maps clean_ticker over tickers. (แยก ticker ออกมาทีละ 1 digit)
        """
        return [clean_string(x) for x in strings]

In [23]:
def create_main_model( types):
    data = ""
    t0 = time()
    a = []
    i = 0
    
    df = joblib.load(master_data)
    print("download data...") 
       
    df = df[[types, 'description']]   
        
       
    df.columns = ['target', 'data']
    n = df.isnull().sum()
    print("missing values : ", n )
    df.dropna(inplace=True)
    df['target'] = df['target'].apply(int)
    print("Load Data.. "+types+" success")
        
        #save the data train
    print("Save data train")
    create_model(df,types)
    
    load_time = time() - t0
    print("Load dataset time:  %0.3fs" % load_time)


In [24]:
def create_data( sheets, types):
    data = ""
    t0 = time()
    a = []
    i = 0
    for s in sheets:
        df = joblib.load('data/train/df_master_train.pkl')
        print("download data...")
        df = df[df['section'] ==s]
       
        df = df[[types, 'description']]
       # df['description'] = df['description'].str.lower()
        
       
        df.columns = ['target', 'data']
        n = df.isnull().sum()
        print("missing values : ", n )
        df.dropna(inplace=True)
        df['target'] = df['target'].apply(int)
        print("Load Data.. "+s+" success")
        
        #save the data train
        print("Save data train")

        joblib.dump(df, 'data/train/df_master_'+s +'.pkl', compress=1)
        print("Save datatable.. "+s+" success")

    
        create_model(df,s)
    
    load_time = time() - t0
    print("Load dataset time:  %0.3fs" % load_time)

In [25]:
types = "section"

create_main_model( types)

download data...
missing values :  target    0
data      1
dtype: int64
Load Data.. section success
Save data train
pipeline of feature engineering and model
paramater selection
Training: 
0.7692227342803772
{'tfidf__use_idf': True, 'vectorizer__ngram_range': (1, 2)}
0.8861183290025985
Save Model
Create Model.. section success
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       105
           2       0.98      0.99      0.98        93
           3       0.94      1.00      0.97        30
           4       0.97      0.97      0.97       157
           5       0.79      0.82      0.81       164
           6       0.88      0.89      0.88       716
           7       0.82      0.87      0.84      2056
           8       0.86      0.85      0.86        60
           9       0.94      0.90      0.92        51
          10       0.89      0.78      0.83       131
          11       0.93      0.90      0.91       565
          12       0.9

In [20]:
import joblib

def test_model(txt):
    print(txt)
    model_main = joblib.load('data/model/model_section.pkl')
    products = txt  # Video game consoles : 95
    p = model_main.predict([products])[0] 
    
    if p <10:
        a = str(p)
        a = ("0"+a)
    else:
        a = str(p)
    print(a)
    model_2 = joblib.load('data/model_2/model_'+a+'.pkl')
    predicts = model_2.predict([products])[0]  
    print(predicts)
    return predicts

In [28]:
txt = "TRANE HIGHT TEMP GREASE"  # FISH OIL - 03 - 1504

test_model(txt)

TRANE HIGHT TEMP GREASE
05
2707


2707